In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Web Scraping Indeed Portal
def scrap_indeed():
    url = 'https://pk.indeed.com/jobs?q=data+analyst&l=Pakistan'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                      '(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
    }

    response = requests.get(url, headers=headers)
    print("Status Code:", response.status_code)
    print("Page Snippet:\n", response.text[:500])

    soup = BeautifulSoup(response.content, 'html.parser')
    jobs = []

    job_cards = soup.find_all('a', class_='tapItem')
    print("Found job cards:", len(job_cards))

    for job_elem in job_cards:
        title = job_elem.find('h2', class_='jobTitle')
        company = job_elem.find('span', class_='companyName')
        location = job_elem.find('div', class_='companyLocation')
        date = job_elem.find('span', class_='date')

        job_data = [
            title.text.strip() if title else 'N/A',
            company.text.strip() if company else 'N/A',
            location.text.strip() if location else 'N/A',
            date.text.strip() if date else 'N/A'
        ]
        print(job_data)
        jobs.append(job_data)

    if jobs:
        df = pd.DataFrame(jobs, columns=['Title', 'Company', 'Location', 'Date Posted'])
        df.to_csv('indeed_jobs.csv', index=False)
        print("CSV file saved.")
    else:
        print("No jobs found.")

# Call the function to run it
scrap_indeed()


Status Code: 403
Page Snippet:
 <!DOCTYPE html><html lang="en"><head><meta charset="utf-8"><title>Security Check - Indeed.com</title><meta name="viewport" content="width=device-width, initial-scale=1"><style>:root{color-scheme:light dark;--background-color:#fff;--primary-1000:#0d2d5e;--primary-900:#164081;--primary-800:#2557a7;--primary-700:#3f73d3;--primary-600:#6792f0;--neutral-1000:#2d2d2d;--neutral-900:#424242;--neutral-400:#d4d2d0;--dark-1000:#040606;--link-color:var(--primary-800);--link-color-hover:var(--primary-900);--
Found job cards: 0
No jobs found.


In [12]:
#Web Scraping Indeed.com
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = 'https://pk.indeed.com/jobs?q=data+analyst&l=Pakistan'

headers = {
    "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Mobile Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
}

# Updated cookie string
cookies_str = '''CTK=1in4pqkrv230i001; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtfyXg82hUewalme7ENIhkuI="; CSRF=6IxTkSeCuGUChaieKZgdE1EQi6UbCGz1; INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; SURF=dydG5Jw0lg9TUpcX28sEkNnUyHTHLoDk; _ga=GA1.1.1379756336.1748113943; FPID=FPID2.2.HEywW5YKmm%2BCkrRMBSiZFOYeRoA0pkgZGtob4TtgS%2Bk%3D.1748113943; FPLC=y1XcRcBM9ccvQsHSrJMUF0tKs8cW%2FO%2F1LUcHdnRZFIroOcl9gOMQ2w4ajHAfrt4WfVW7OiZFlRBHph%2Fn1WrZtzXELCDl7rsWpDbW9ocqIRSUohT4IK37%2B26kud6AuQ%3D%3D; LC="co=PK"; indeed_rcc=PREF:LV:CTK; SHARED_INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; LOCALE=en_PK; MICRO_CONTENT_CSRF_TOKEN=DJJj4RjVHLvOpHB98DV9zd0GNkD4JTCe; PPID=""; PREF="TM=1748114192406:L=Pakistan"; LV=LA=1748126556:LV=1748119843:CV=1748126556:TS=1748114013; PP=1; appDownloadPromoFirstPage=serp; appDownloadPromoSecondPage=viewjob; _MAGUA_SsnClsdCnt=2; _MAGUA_MnthClsdCnt=2; _MAGUA_ClsdPgs=serp%20viewjob; ROJC=997206b0cfc69c44:3eddd874ec49075c:c0fa390c08522e10; _MAGUA_SprssG1T=0; NOMOB=1; _cfuvid=_9V9SAAJalcGdw2_6mqCAGr3bybudG3Xje4zIVXPVV0-1748128775439-0.0.1.1-604800000; RQ=q=data+analyst&l=Pakistan&ts=1748128842792&pts=1748119843312:q=data+analyst&l=karachi&ts=1748114132299; JSESSIONID=node01icj44b5q8ipd1m53dexqcb43619911952.node0; _ga_LYNT3BTHPG=GS2.1.s1748126042$o3$g1$t1748128785$j0$l0$h782139064; __cf_bm=7JpVYMjw1vHThX_iJo0M0oJVsmHjbvYOe3RlN6J3Tls-1748128860-1.0.1.1-t0VeA43ahXGs0MjirkREJbqBlVuJGaw5iV6Ev8O0bR3o5Y3q5K6uJ8Jzn3ComLAxj3iH6tFjHklbdZD2O8GjXE5cTEZyIQmb.6g_uNkj.nM; cf_clearance=ly9xeSMByXNQsFhEKR92I.WXcgbArAzJdTE0h8M.CKo-1748128873-1.2.1.1-jwQEMY0e9n0cxdYY7Rfmd7H_JDagFyMTEjJ42n6TdawVxjHHkM.41i3.Zj9wqo9P2mWmgZZn7LQvMqTmQWeSUL1z0rVsl7ITWnGga45Z85mAF9b7tv..jW_hFKdttqOUsDt1pikP1IDkJMqWMTKCJB20CXymZD8swJ0F15ZgWxRb.ZVovud7rXke7EBgWtDU1QWSA1T.E905Q4rCTPE4wtUKCtwAQQzOs371_NlZZi0ramifbfcR9B8pSylpwUhLNRn5qB4.GlnQhac4Q5M1xlAiiRYpbxCf4GRsKNkgrr0B3ucr2FRUptwypoVwIjlWpLtmmU7ow5SgxSqK8ZzZbUPBUYfhI0JCInX7eDPa3waZ5KFDt1ckFkPIK4kAjxoE'''

# Convert cookie string to dictionary
cookies = dict([cookie.strip().split('=', 1) for cookie in cookies_str.split(';')])

response = requests.get(url, headers=headers, cookies=cookies)
time.sleep(2)

soup = BeautifulSoup(response.content, 'html.parser')

job_cards = soup.select('a.tapItem, div.job_seen_beacon')
print("Found job cards:", len(job_cards))

jobs = []
for i, job_elem in enumerate(job_cards):
    title_elem = job_elem.find('h2')
    company_elem = job_elem.find('span', string=True)
    location_elem = job_elem.find('div', class_='companyLocation')
    date_elem = job_elem.find('span', class_='date')

    title = title_elem.get_text(strip=True) if title_elem else 'N/A'
    company = company_elem.get_text(strip=True) if company_elem else 'N/A'
    location = location_elem.get_text(strip=True) if location_elem else 'N/A'
    date = date_elem.get_text(strip=True) if date_elem else 'N/A'

    if company == 'N/A' or location == 'N/A' or date == 'N/A':
        print(f"\n Job card HTML for index {i}:\n", job_elem.prettify())

    jobs.append([title, company, location, date])

if jobs:
    df = pd.DataFrame(jobs, columns=['Title', 'Company', 'Location', 'Date Posted'])
    df.to_csv('indeed_jobs_debug.csv', index=False)
    print("CSV saved.")
else:
    print(" No jobs found.")

Found job cards: 25

🔍 Job card HTML for index 0:
 <div class="job_seen_beacon">
 <table cellpadding="0" cellspacing="0" class="mainContentTable css-1v69ab7 eu4oa1w0" role="presentation">
  <tbody>
   <tr>
    <td class="resultContent css-1o6lhys eu4oa1w0">
     <div class="css-pt3vth e37uo190">
      <h2 class="jobTitle css-1psdjh5 eu4oa1w0" tabindex="-1">
       <a aria-label="full details of Technical Data Analyst" class="jcs-JobTitle css-1baag51 eu4oa1w0" data-hide-spinner="true" data-hiring-event="false" data-jk="3eddd874ec49075c" data-mobtk="1is2ad2o229ab06r" href="/viewjob?jk=3eddd874ec49075c" id="job_3eddd874ec49075c" target="_blank">
        Technical Data Analyst
       </a>
      </h2>
      <div class="css-uhbet5 e37uo190">
      </div>
     </div>
     <div class="company_location css-i375s1 e37uo190">
      <div data-testid="timing-attribute" elementtiming="significant-render">
       <div class="css-1afmp4o e37uo190">
        <span class="css-1h7lukg eu4oa1w0" data-testi

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = 'https://pk.indeed.com/jobs?q=data+analyst&l=Pakistan'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

# Updated cookie string
cookies_str = '''CTK=1in4pqkrv230i001; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtfyXg82hUewalme7ENIhkuI="; CSRF=6IxTkSeCuGUChaieKZgdE1EQi6UbCGz1; INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; SURF=dydG5Jw0lg9TUpcX28sEkNnUyHTHLoDk; _ga=GA1.1.1379756336.1748113943; FPID=FPID2.2.HEywW5YKmm%2BCkrRMBSiZFOYeRoA0pkgZGtob4TtgS%2Bk%3D.1748113943; FPLC=y1XcRcBM9ccvQsHSrJMUF0tKs8cW%2FO%2F1LUcHdnRZFIroOcl9gOMQ2w4ajHAfrt4WfVW7OiZFlRBHph%2Fn1WrZtzXELCDl7rsWpDbW9ocqIRSUohT4IK37%2B26kud6AuQ%3D%3D; LC="co=PK"; indeed_rcc=PREF:LV:CTK; SHARED_INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; LOCALE=en_PK; MICRO_CONTENT_CSRF_TOKEN=DJJj4RjVHLvOpHB98DV9zd0GNkD4JTCe; PPID=""; PREF="TM=1748114192406:L=Pakistan"; PP=1; appDownloadPromoFirstPage=serp; appDownloadPromoSecondPage=viewjob; _MAGUA_SsnClsdCnt=2; _MAGUA_MnthClsdCnt=2; _MAGUA_ClsdPgs=serp%20viewjob; ROJC=997206b0cfc69c44:3eddd874ec49075c:c0fa390c08522e10; _MAGUA_SprssG1T=0; NOMOB=1; LV=LA=1748130245:LV=1748126556:CV=1748130245:TS=1748114013; cf_clearance=fdA5nFG1ns3hUhGMsyoW2MNAYHjoXaMn8ozZrPfKR34-1748130442-1.2.1.1-JXFXv37QeX5XsVvGLA7YfPSk5Xu0HFHgqDZjptXiopY9mbjGm.zm08ZV8rgD.aHRhF4CIG9j3y4lReFV8nRePxcBTxC7f7LUrOQofIHt7Pdxn4AYElGmI7Zakjt0nq6eYVnLYB5V3esCH0lUvJvzVCxhGku0pt647eHF.zk0BIb8TY.hrRQkwXzm3392Nogbxcpgras13P4.GshCHvosRJkgA9WGm.W5Y73._4O0V4aERERXRRWAc9s.RA7Sz3eTrs.oYYzB6kbmnFuHonIIsaWpndbc0rBAseMorfsCH9TxtcdFz1neGtho4u0pemfzNp6G9xb2xt395yrcjf6EQSXMohKLCDJ8Rg_FKjbCJkaLJSUqtbKqv56L4Dyt0YWu; RQ=q=data+analyst&l=Pakistan&ts=1748130442545&pts=1748119843312:q=data+analyst&l=karachi&ts=1748114132299; JSESSIONID=node09wglsa89jy5fbefwpbrvvaof8690157.node0; __cf_bm=lGWycwgBvSp8x2E58R88ZdEv6tL.y9BSGIg46xU5n_Q-1748130444-1.0.1.1-pkaBDGYMWbe_6f0FTNahMZiGxhQlVOtx.WcJ6otbRzoy2YUn9PLiNpX2.eVgiWXjijutEU1ul5z8TwYusUodyYHhKBkJ6lCgZcEonfB_Cm0; _cfuvid=daD8jVKFGNWrN4OW09VCJaC2oLFm4OvVrF6ygkZ4x9k-1748130444117-0.0.1.1-604800000; _ga_LYNT3BTHPG=GS2.1.s1748126042$o3$g1$t1748130370$j0$l0$h782139064'''
#CTK=1in4pqkrv230i001; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtfyXg82hUewalme7ENIhkuI="; CSRF=6IxTkSeCuGUChaieKZgdE1EQi6UbCGz1; INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; SURF=dydG5Jw0lg9TUpcX28sEkNnUyHTHLoDk; _ga=GA1.1.1379756336.1748113943; FPID=FPID2.2.HEywW5YKmm%2BCkrRMBSiZFOYeRoA0pkgZGtob4TtgS%2Bk%3D.1748113943; FPLC=y1XcRcBM9ccvQsHSrJMUF0tKs8cW%2FO%2F1LUcHdnRZFIroOcl9gOMQ2w4ajHAfrt4WfVW7OiZFlRBHph%2Fn1WrZtzXELCDl7rsWpDbW9ocqIRSUohT4IK37%2B26kud6AuQ%3D%3D; LC="co=PK"; indeed_rcc=PREF:LV:CTK; SHARED_INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; LOCALE=en_PK; MICRO_CONTENT_CSRF_TOKEN=DJJj4RjVHLvOpHB98DV9zd0GNkD4JTCe; PPID=""; PREF="TM=1748114192406:L=Pakistan"; PP=1; appDownloadPromoFirstPage=serp; appDownloadPromoSecondPage=viewjob; _MAGUA_SsnClsdCnt=2; _MAGUA_MnthClsdCnt=2; _MAGUA_ClsdPgs=serp%20viewjob; ROJC=997206b0cfc69c44:3eddd874ec49075c:c0fa390c08522e10; _MAGUA_SprssG1T=0; NOMOB=1; LV=LA=1748130245:LV=1748126556:CV=1748130245:TS=1748114013; RQ=q=data+analyst&l=Pakistan&ts=1748130395793&pts=1748119843312:q=data+analyst&l=karachi&ts=1748114132299; _cfuvid=iIIdpi5HwR3Rkhy81CYFWsDrXmCZNGaJbCQZ2kCBEfI-1748130396244-0.0.1.1-604800000; JSESSIONID=node01tpkqrl3quihi7sgb99loboeu19940614.node0; _ga_LYNT3BTHPG=GS2.1.s1748126042$o3$g1$t1748130356$j0$l0$h782139064; __cf_bm=x8qTea_tdMOuHvy8izPkRIDc1J22LN_XWAk.zmZ9zUw-1748130431-1.0.1.1-15.p8tDwEJnXnx67o3vHy9RbtmgpzTnTLcCQzxyw9aE9CnDFyM6MmUTK_.W4afC7DJm7zUKtqbJ8u5CQX1PA3zODkmWLUMXpKpQfAIXY7J4; cf_clearance=fdA5nFG1ns3hUhGMsyoW2MNAYHjoXaMn8ozZrPfKR34-1748130442-1.2.1.1-JXFXv37QeX5XsVvGLA7YfPSk5Xu0HFHgqDZjptXiopY9mbjGm.zm08ZV8rgD.aHRhF4CIG9j3y4lReFV8nRePxcBTxC7f7LUrOQofIHt7Pdxn4AYElGmI7Zakjt0nq6eYVnLYB5V3esCH0lUvJvzVCxhGku0pt647eHF.zk0BIb8TY.hrRQkwXzm3392Nogbxcpgras13P4.GshCHvosRJkgA9WGm.W5Y73._4O0V4aERERXRRWAc9s.RA7Sz3eTrs.oYYzB6kbmnFuHonIIsaWpndbc0rBAseMorfsCH9TxtcdFz1neGtho4u0pemfzNp6G9xb2xt395yrcjf6EQSXMohKLCDJ8Rg_FKjbCJkaLJSUqtbKqv56L4Dyt0YWu'''

# Convert cookie string to dictionary
cookies = dict([cookie.strip().split('=', 1) for cookie in cookies_str.split(';')])

response = requests.get(url, headers=headers, cookies=cookies)
time.sleep(3)

soup = BeautifulSoup(response.content, 'html.parser')
job_cards = soup.select('a.tapItem, div.job_seen_beacon')
print("Found job cards:", len(job_cards))

jobs = []
for i, job_elem in enumerate(job_cards):
    title_elem = job_elem.find('h2')
    company_elem = job_elem.find('span', {'class': 'companyName'})

    # Try multiple methods to extract location
    location_elem = (
        job_elem.find('div', class_='companyLocation') or
        job_elem.find('div', class_='company_location') or
        job_elem.find('div', class_='metadata') or
        job_elem.find('span', string=lambda x: x and ('location' in x.lower()))
    )

    # Try multiple methods to extract date posted
    date_elem = (
        job_elem.find('span', class_='date') or
        job_elem.find('span', class_='dateLabel') or
        job_elem.find('span', string=lambda x: x and ('day' in x.lower() or 'ago' in x.lower()))
    )

    title = title_elem.get_text(strip=True) if title_elem else 'N/A'
    company = company_elem.get_text(strip=True) if company_elem else 'N/A'
    location = location_elem.get_text(strip=True) if location_elem else 'N/A'
    date = date_elem.get_text(strip=True) if date_elem else 'N/A'

    if location == 'N/A' or date == 'N/A':
        print(f"\n Missing info at index {i}. Job card HTML:\n", job_elem.prettify())

    jobs.append([title, company, location, date])

if jobs:
    df = pd.DataFrame(jobs, columns=['Title', 'Company', 'Location', 'Date Posted'])
    df.to_csv('indeed_jobs_debug.csv', index=False)
    print(" CSV saved with updated fields.")
else:
    print(" No jobs found.")

Found job cards: 0
⚠️ No jobs found.


In [2]:
#Web Scraping of Glassdoor.com for Data Collection
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#Web Scraping of GlassDoor 
def parse_cookie_string(cookie_str):
    return dict([cookie.strip().split('=', 1) for cookie in cookie_str.split(';') if '=' in cookie])


# url for one specific job
url = 'https://www.glassdoor.com/Job/graphic-designer-jobs-SRCH_KO0,16.htm'

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
        "(KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
    )
}

cookie_string = '''
gdId=04ab8b23-c429-4141-9840-0fa2980bf5f3; rl_page_init_referrer=RudderEncrypt%3AU2FsdGVkX19WdTayebDvx7Px%2FNV56nBOsH0SahlH7AZsJMJe7TmWdwo3%2FqxWm0z1; rl_page_init_referring_domain=RudderEncrypt%3AU2FsdGVkX1%2BpybW0RSQaDjij51dte830aZoIsiF22ai36BV7cGKWQ1NkSU%2FxvVYq; _optionalConsent=true; gdsid=1748114292883:1748131461415:2C2D8B67A9D4C1F8777AF8EABE381B6D; asst=1748131461.2; alr=https%3A%2F%2Fwww.google.com%2F; __cf_bm=0Pqsc7Ulda36LLukof6hHXJRahLgm_nOXl4k_M7h4vA-1748131461-1.0.1.1-P55XFWvOgGeTTO6rqvY0uM4A2Uk6owdgYT6.I7RccRa2_jT4v7aE66c1Ez9kUWqf87LPmNIdn5c1wHW9tFJrJBV3NG5TlXgs6.roZSmPjopn871nwmHrcGJYflpGGofP; _cfuvid=0KxrA1uxVxdGT6kd7OB2ignDiD6ptb5TTVlUybkRqcA-1748131461911-0.0.1.1-604800000; GSESSIONID=undefined; cf_clearance=Q3kyy2bjmVvwesKMt8_9q_0G72j5YFLjDAH2V4feH9c-1748131464-1.2.1.1-ppGRaooq1GDBvs1PZg2.bNIk_ClMD8JKSdji0hf7l8H80OELP00RL3sGG7gZ5skUHQZOpQFJuy29ZDAHPA7MzV7KgBnN7gzG2Jpw2YDvoQ01CK8h5p7bCGZ05nmvwpnoipzlZORP_pVHvxGGv2bY0Q0ZIVIyVTp9pTj4d_aq4QSYy3uUZ3EEu8wb.U.ma8Uyn6pZv.0NdTyzSTOvsHrnls.8yV2zek8sKjHjCI9HTxStsWloaCp7LctQBwITEa7O.0kS4DD.AHlwIHreqr2cpOfWSvUOETAwWbJdzywCGMYP5_GLqiTx1L8iD74S8r8a8Y5iy7Cb_qYwRyERiYYL1BUIGmQ.EfsPnWZl3.3C4L4; rsSessionId=1748131380240; AWSALB=IFmVfhYEjUQW3XVcsOt/GmIPm/Zy505l6gOoKs8DLk7RBzBw28dOoE4lyZI1L8VXIalJcYzNQWz8f3r7sn5xdg9VotT12DBJdQyZydi4bqOULUGfspJKwdptStaV; AWSALBCORS=IFmVfhYEjUQW3XVcsOt/GmIPm/Zy505l6gOoKs8DLk7RBzBw28dOoE4lyZI1L8VXIalJcYzNQWz8f3r7sn5xdg9VotT12DBJdQyZydi4bqOULUGfspJKwdptStaV; JSESSIONID=2B251CABF5E9F2AD1D7E53D4A703B0CA; cass=0; cdzNum=1; indeedCtk=1is2c2lu3kuor800; rl_user_id=RudderEncrypt%3AU2FsdGVkX1%2BFGk%2FwPer8gM4esLVaw1gDAVvmWu3RRmA%3D; rl_trait=RudderEncrypt%3AU2FsdGVkX1%2BlnjtxHAR4lnRd%2F4jJLFt%2BizmL%2F66YgR8%3D; rl_group_id=RudderEncrypt%3AU2FsdGVkX1%2B1uUMWUexiRmtidl5tWEr2m29FzqHLea0%3D; rl_group_trait=RudderEncrypt%3AU2FsdGVkX18QElAmKZFGL0333acY%2BRwdze7Dj5sO8TE%3D; cdArr=; OptanonConsent=isGpcEnabled=0&datestamp=Sun+May+25+2025+05%3A03%3A19+GMT%2B0500+(Pakistan+Standard+Time)&version=202407.2.0&browserGpcFlag=0&isIABGlobal=false&hosts=&consentId=d3cefeb9-aea5-4e18-bb49-4f3ed8693aea&interactionCount=1&isAnonUser=1&landingPath=NotLandingPage&groups=C0001%3A1%2CC0003%3A1%2CC0002%3A1%2CC0004%3A0%2CC0017%3A1&AwaitingReconsent=false; rsReferrerData=%7B%22currentPageRollup%22%3A%22%2Fjob%2Fjobs-srch%22%2C%22previousPageRollup%22%3A%22%2Fjob%2Fjobs-srch%22%2C%22currentPageAbstract%22%3A%22%2FJob%2F%5BOCC%5D-jobs-SRCH_%5BPRM%5D.htm%22%2C%22previousPageAbstract%22%3A%22%2FJob%2F%5BOCC%5D-jobs-SRCH_%5BPRM%5D.htm%22%2C%22currentPageFull%22%3A%22https%3A%2F%2Fwww.glassdoor.com%2FJob%2Fgraphic-designer-jobs-SRCH_KO0%2C16.htm%22%2C%22previousPageFull%22%3A%22https%3A%2F%2Fwww.glassdoor.com%2FJob%2Fgraphic-designer-jobs-SRCH_KO0%2C16.htm%22%7D; rl_anonymous_id=RudderEncrypt%3AU2FsdGVkX1%2BOBVuHTtvbTeGGb9HbO3exalHhOoCY%2B0bCSi1B5FgMMBZfMJhdm9B8q8ae4zOhWtGumspSMYVGjg%3D%3D; rl_session=RudderEncrypt%3AU2FsdGVkX1%2BqMeTEX2tHhdYjBsGNfjOzcb8OzW%2BThzEVa07jlIKdVxPQL%2Br5NJ6tHMVaJ7v9ZzNxGY8Rd%2F7JqHnhwxtmGcOzYhalgyR2EjG713yakLZQy7QSy%2Fw%2FFaOVYSvD7LJKp61j7tKkO8WQeQ%3D%3D; bs=J7ujuHKXoa1Busl4Fj9nHg:J9ahTrnq6b7A2sGIQbiRPC0M3mXIxzP_J7w-GqCGHPVd8NEvUT95ue3v_9aJu18vY1jtLUiJkw8NicT40VQxI8ILbQ1QyF5Nd_IiYxw0akA:lvGiairwbk9NsrFKxpH7Rwtnpv78_e2tuYHoBNXAPjc; _dd_s=rum=0&expire=1748132518157'''


#CTK=1in4pqkrv230i001; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtfyXg82hUewalme7ENIhkuI="; CSRF=6IxTkSeCuGUChaieKZgdE1EQi6UbCGz1; INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; SURF=dydG5Jw0lg9TUpcX28sEkNnUyHTHLoDk; _ga=GA1.1.1379756336.1748113943; FPID=FPID2.2.HEywW5YKmm%2BCkrRMBSiZFOYeRoA0pkgZGtob4TtgS%2Bk%3D.1748113943; FPLC=y1XcRcBM9ccvQsHSrJMUF0tKs8cW%2FO%2F1LUcHdnRZFIroOcl9gOMQ2w4ajHAfrt4WfVW7OiZFlRBHph%2Fn1WrZtzXELCDl7rsWpDbW9ocqIRSUohT4IK37%2B26kud6AuQ%3D%3D; LC="co=PK"; ...'''

cookies = parse_cookie_string(cookie_string)

# Send HTTP request
try:
    response = requests.get(url, headers=headers, cookies=cookies)
    response.raise_for_status()
except requests.RequestException as e:
    print(f" Request failed: {e}")
    exit()

# Sleep to mimic human behavior
time.sleep(2)

# Parse HTML
soup = BeautifulSoup(response.content, 'html.parser')

# Find job cards
job_cards = soup.select('a.tapItem, div.job_seen_beacon')
print(f" Found {len(job_cards)} job cards.")

jobs = []

for i, card in enumerate(job_cards):
    title_elem = card.find('h2')
    company_elem = card.find('span', class_='companyName')

    location_elem = (
        card.find('div', class_='companyLocation') or
        card.find('div', class_='company_location') or
        card.find('div', class_='metadata') or
        card.find('span', string=lambda x: x and 'location' in x.lower())
    )

    date_elem = (
        card.find('span', class_='date') or
        card.find('span', class_='dateLabel') or
        card.find('span', string=lambda x: x and ('day' in x.lower() or 'ago' in x.lower()))
    )

    title = title_elem.get_text(strip=True) if title_elem else 'N/A'
    company = company_elem.get_text(strip=True) if company_elem else 'N/A'
    location = location_elem.get_text(strip=True) if location_elem else 'N/A'
    date_posted = date_elem.get_text(strip=True) if date_elem else 'N/A'

    if location == 'N/A' or date_posted == 'N/A':
        print(f"\n Missing data at index {i}. Card HTML snippet:\n{card.prettify()[:500]}...\n")

    jobs.append([title, company, location, date_posted])

# Export to CSV
if jobs:
    df = pd.DataFrame(jobs, columns=['Title', 'Company', 'Location', 'Date Posted'])
    df.to_csv('glassdoor_graphic_designer_jobs.csv', index=False)
    print(" CSV saved: glassdoor_graphic_designer_jobs.csv")
else:
    print("No job listings extracted.")

❌ Request failed: 403 Client Error: Forbidden for url: https://www.glassdoor.com/Job/graphic-designer-jobs-SRCH_KO0,16.htm
🔍 Found 0 job cards.
⚠️ No job listings extracted.


In [2]:
#Web Scraping of Rozee.Pk for Data Collection
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

url = 'https://www.rozee.pk/job/jsearch/q/software%20engineer'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
}

# Updated cookie string
cookies_str = '''CTK=1in4pqkrv230i001; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtfyXg82hUewalme7ENIhkuI="; CSRF=6IxTkSeCuGUChaieKZgdE1EQi6UbCGz1; INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; SURF=dydG5Jw0lg9TUpcX28sEkNnUyHTHLoDk; _ga=GA1.1.1379756336.1748113943; FPID=FPID2.2.HEywW5YKmm%2BCkrRMBSiZFOYeRoA0pkgZGtob4TtgS%2Bk%3D.1748113943; FPLC=y1XcRcBM9ccvQsHSrJMUF0tKs8cW%2FO%2F1LUcHdnRZFIroOcl9gOMQ2w4ajHAfrt4WfVW7OiZFlRBHph%2Fn1WrZtzXELCDl7rsWpDbW9ocqIRSUohT4IK37%2B26kud6AuQ%3D%3D; LC="co=PK"; indeed_rcc=PREF:LV:CTK; SHARED_INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; LOCALE=en_PK; MICRO_CONTENT_CSRF_TOKEN=DJJj4RjVHLvOpHB98DV9zd0GNkD4JTCe; PPID=""; PREF="TM=1748114192406:L=Pakistan"; PP=1; appDownloadPromoFirstPage=serp; appDownloadPromoSecondPage=viewjob; _MAGUA_SsnClsdCnt=2; _MAGUA_MnthClsdCnt=2; _MAGUA_ClsdPgs=serp%20viewjob; ROJC=997206b0cfc69c44:3eddd874ec49075c:c0fa390c08522e10; _MAGUA_SprssG1T=0; NOMOB=1; LV=LA=1748130245:LV=1748126556:CV=1748130245:TS=1748114013; cf_clearance=fdA5nFG1ns3hUhGMsyoW2MNAYHjoXaMn8ozZrPfKR34-1748130442-1.2.1.1-JXFXv37QeX5XsVvGLA7YfPSk5Xu0HFHgqDZjptXiopY9mbjGm.zm08ZV8rgD.aHRhF4CIG9j3y4lReFV8nRePxcBTxC7f7LUrOQofIHt7Pdxn4AYElGmI7Zakjt0nq6eYVnLYB5V3esCH0lUvJvzVCxhGku0pt647eHF.zk0BIb8TY.hrRQkwXzm3392Nogbxcpgras13P4.GshCHvosRJkgA9WGm.W5Y73._4O0V4aERERXRRWAc9s.RA7Sz3eTrs.oYYzB6kbmnFuHonIIsaWpndbc0rBAseMorfsCH9TxtcdFz1neGtho4u0pemfzNp6G9xb2xt395yrcjf6EQSXMohKLCDJ8Rg_FKjbCJkaLJSUqtbKqv56L4Dyt0YWu; RQ=q=data+analyst&l=Pakistan&ts=1748130442545&pts=1748119843312:q=data+analyst&l=karachi&ts=1748114132299; JSESSIONID=node09wglsa89jy5fbefwpbrvvaof8690157.node0; __cf_bm=lGWycwgBvSp8x2E58R88ZdEv6tL.y9BSGIg46xU5n_Q-1748130444-1.0.1.1-pkaBDGYMWbe_6f0FTNahMZiGxhQlVOtx.WcJ6otbRzoy2YUn9PLiNpX2.eVgiWXjijutEU1ul5z8TwYusUodyYHhKBkJ6lCgZcEonfB_Cm0; _cfuvid=daD8jVKFGNWrN4OW09VCJaC2oLFm4OvVrF6ygkZ4x9k-1748130444117-0.0.1.1-604800000; _ga_LYNT3BTHPG=GS2.1.s1748126042$o3$g1$t1748130370$j0$l0$h782139064'''
#CTK=1in4pqkrv230i001; RF="TFTzyBUJoNr6YttPP3kyivpZ6-9J49o-Uk3iY6QNQqKE2fh7FyVgtfyXg82hUewalme7ENIhkuI="; CSRF=6IxTkSeCuGUChaieKZgdE1EQi6UbCGz1; INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; SURF=dydG5Jw0lg9TUpcX28sEkNnUyHTHLoDk; _ga=GA1.1.1379756336.1748113943; FPID=FPID2.2.HEywW5YKmm%2BCkrRMBSiZFOYeRoA0pkgZGtob4TtgS%2Bk%3D.1748113943; FPLC=y1XcRcBM9ccvQsHSrJMUF0tKs8cW%2FO%2F1LUcHdnRZFIroOcl9gOMQ2w4ajHAfrt4WfVW7OiZFlRBHph%2Fn1WrZtzXELCDl7rsWpDbW9ocqIRSUohT4IK37%2B26kud6AuQ%3D%3D; LC="co=PK"; indeed_rcc=PREF:LV:CTK; SHARED_INDEED_CSRF_TOKEN=cmvUVdaqL0F8lYozm9b9Ldsy4WwMYgqL; LOCALE=en_PK; MICRO_CONTENT_CSRF_TOKEN=DJJj4RjVHLvOpHB98DV9zd0GNkD4JTCe; PPID=""; PREF="TM=1748114192406:L=Pakistan"; PP=1; appDownloadPromoFirstPage=serp; appDownloadPromoSecondPage=viewjob; _MAGUA_SsnClsdCnt=2; _MAGUA_MnthClsdCnt=2; _MAGUA_ClsdPgs=serp%20viewjob; ROJC=997206b0cfc69c44:3eddd874ec49075c:c0fa390c08522e10; _MAGUA_SprssG1T=0; NOMOB=1; LV=LA=1748130245:LV=1748126556:CV=1748130245:TS=1748114013; RQ=q=data+analyst&l=Pakistan&ts=1748130395793&pts=1748119843312:q=data+analyst&l=karachi&ts=1748114132299; _cfuvid=iIIdpi5HwR3Rkhy81CYFWsDrXmCZNGaJbCQZ2kCBEfI-1748130396244-0.0.1.1-604800000; JSESSIONID=node01tpkqrl3quihi7sgb99loboeu19940614.node0; _ga_LYNT3BTHPG=GS2.1.s1748126042$o3$g1$t1748130356$j0$l0$h782139064; __cf_bm=x8qTea_tdMOuHvy8izPkRIDc1J22LN_XWAk.zmZ9zUw-1748130431-1.0.1.1-15.p8tDwEJnXnx67o3vHy9RbtmgpzTnTLcCQzxyw9aE9CnDFyM6MmUTK_.W4afC7DJm7zUKtqbJ8u5CQX1PA3zODkmWLUMXpKpQfAIXY7J4; cf_clearance=fdA5nFG1ns3hUhGMsyoW2MNAYHjoXaMn8ozZrPfKR34-1748130442-1.2.1.1-JXFXv37QeX5XsVvGLA7YfPSk5Xu0HFHgqDZjptXiopY9mbjGm.zm08ZV8rgD.aHRhF4CIG9j3y4lReFV8nRePxcBTxC7f7LUrOQofIHt7Pdxn4AYElGmI7Zakjt0nq6eYVnLYB5V3esCH0lUvJvzVCxhGku0pt647eHF.zk0BIb8TY.hrRQkwXzm3392Nogbxcpgras13P4.GshCHvosRJkgA9WGm.W5Y73._4O0V4aERERXRRWAc9s.RA7Sz3eTrs.oYYzB6kbmnFuHonIIsaWpndbc0rBAseMorfsCH9TxtcdFz1neGtho4u0pemfzNp6G9xb2xt395yrcjf6EQSXMohKLCDJ8Rg_FKjbCJkaLJSUqtbKqv56L4Dyt0YWu'''

# Convert cookie string to dictionary
cookies = dict([cookie.strip().split('=', 1) for cookie in cookies_str.split(';')])

response = requests.get(url, headers=headers)
time.sleep(3)

soup = BeautifulSoup(response.content, 'html.parser')
job_cards = soup.select('a.tapItem, div.job_seen_beacon')
print("Found job cards:", len(job_cards))

jobs = []
for i, job_elem in enumerate(job_cards):
    title_elem = job_elem.find('h2')
    company_elem = job_elem.find('span', {'class': 'companyName'})

    # Try multiple methods to extract location
    location_elem = (
        job_elem.find('div', class_='companyLocation') or
        job_elem.find('div', class_='company_location') or
        job_elem.find('div', class_='metadata') or
        job_elem.find('span', string=lambda x: x and ('location' in x.lower()))
    )

    # Try multiple methods to extract date posted
    date_elem = (
        job_elem.find('span', class_='date') or
        job_elem.find('span', class_='dateLabel') or
        job_elem.find('span', string=lambda x: x and ('day' in x.lower() or 'ago' in x.lower()))
    )

    title = title_elem.get_text(strip=True) if title_elem else 'N/A'
    company = company_elem.get_text(strip=True) if company_elem else 'N/A'
    location = location_elem.get_text(strip=True) if location_elem else 'N/A'
    date = date_elem.get_text(strip=True) if date_elem else 'N/A'

    if location == 'N/A' or date == 'N/A':
        print(f"\n Missing info at index {i}. Job card HTML:\n", job_elem.prettify())

    jobs.append([title, company, location, date])

if jobs:
    df = pd.DataFrame(jobs, columns=['Title', 'Company', 'Location', 'Date Posted'])
    df.to_csv('rozee_jobs_.csv', index=False)
    print(" CSV saved with updated fields.")
else:
    print( "No jobs found.")

Found job cards: 0
No jobs found.
